In [2]:
!pip install geopy

  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)

   ---------------------------------------- 0/2 [geographiclib]
   ---------------------------------------- 0/2 [geographiclib]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   ---------------------------------------- 2/2 [geopy]



In [3]:
# STEP 1: Imports
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [4]:
# STEP 2: Load & Preprocess Data
df = pd.read_csv("dataset.csv")

df.rename(columns={"SystemCodeNumber": "Lot_ID"}, inplace=True)
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")


In [5]:
# Traffic & vehicle weights
traffic_level_weights = {"low": 1, "average": 2, "high": 3}
vehicle_type_weights = {"car": 1, "bike": 0.8, "truck": 1.2, "cycle": 0.5}

df["Traffic_Level"] = df["TrafficConditionNearby"].map(traffic_level_weights)
df["Vehicle_Weight"] = df["VehicleType"].map(vehicle_type_weights)


In [6]:
# Remove rows with missing data in key columns
df = df.dropna(subset=["Occupancy", "Capacity", "QueueLength", "Traffic_Level", "Vehicle_Weight"])

# STEP 3: Compute Demand Score (like Model 2)
def compute_demand(row, alpha=0.6, beta=0.3, gamma=0.5, delta=0.2, epsilon=0.1):
    occupancy_rate = float(row["Occupancy"]) / float(row["Capacity"])
    return (
        alpha * occupancy_rate +
        beta * row["QueueLength"] +
        gamma * row["Traffic_Level"] +
        delta * row["IsSpecialDay"] +
        epsilon * row["Vehicle_Weight"]
    )

df["Demand_Score"] = df.apply(compute_demand, axis=1)



In [7]:
# STEP 4: Add Distance-Based Competition Analysis
# Function to find nearby lots
def find_nearby_lots(target_row, all_rows, radius_km=0.5):
    target_coords = (target_row["Latitude"], target_row["Longitude"])
    nearby = []

    for idx, row in all_rows.iterrows():
        if row["Lot_ID"] != target_row["Lot_ID"]:
            coords = (row["Latitude"], row["Longitude"])
            if geodesic(target_coords, coords).km <= radius_km:
                nearby.append(row["Demand_Score"])
    return nearby


In [1]:
# MODEL 3: Competitive Pricing with Optimized KDTree Approach

# STEP 1: Imports
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

# Display plots inside the notebook
output_notebook()

# STEP 2: Load and Preprocess Data
df = pd.read_csv("dataset.csv")

# Rename for consistency
df.rename(columns={"SystemCodeNumber": "Lot_ID"}, inplace=True)

# Combine date and time into timestamp
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")

# Map weights
traffic_level_weights = {"low": 1, "average": 2, "high": 3}
vehicle_type_weights = {"car": 1, "bike": 0.8, "truck": 1.2, "cycle": 0.5}

df["Traffic_Level"] = df["TrafficConditionNearby"].map(traffic_level_weights)
df["Vehicle_Weight"] = df["VehicleType"].map(vehicle_type_weights)

# Drop rows with missing data
df.dropna(subset=["Occupancy", "Capacity", "QueueLength", "Traffic_Level", "Vehicle_Weight"], inplace=True)

# STEP 3: Compute Demand Score
def compute_demand(row, alpha=0.6, beta=0.3, gamma=0.5, delta=0.2, epsilon=0.1):
    occupancy_rate = float(row["Occupancy"]) / float(row["Capacity"])
    return (
        alpha * occupancy_rate +
        beta * row["QueueLength"] +
        gamma * row["Traffic_Level"] +
        delta * row["IsSpecialDay"] +
        epsilon * row["Vehicle_Weight"]
    )

df["Demand_Score"] = df.apply(compute_demand, axis=1)

# STEP 4: Efficient Competition-Based Pricing Using KDTree
# Coordinates and spatial index
coords = df[["Latitude", "Longitude"]].to_numpy()
tree = cKDTree(coords)

# Constants
base_price = 20
max_increase = 15
radius_km = 0.5
radius_deg = radius_km / 111  # Approximate conversion from km to degrees

# Normalize range
min_score = df["Demand_Score"].min()
max_score = df["Demand_Score"].max()

# Compute adjusted prices
adjusted_prices = []

for idx, row in df.iterrows():
    # Find nearby lots (within radius_deg)
    nearby_indices = tree.query_ball_point([row["Latitude"], row["Longitude"]], r=radius_deg)
    nearby_indices = [i for i in nearby_indices if i != idx]

    if nearby_indices:
        avg_demand = df.loc[nearby_indices, "Demand_Score"].mean()
    else:
        avg_demand = row["Demand_Score"]

    # Adjusted pricing logic
    adjustment = row["Demand_Score"] - avg_demand
    adjusted_score = row["Demand_Score"] + 0.3 * adjustment
    normalized = (adjusted_score - min_score) / (max_score - min_score)
    price = base_price + normalized * max_increase
    adjusted_prices.append(price)

df["Competitive_Price"] = adjusted_prices

# STEP 5: Visualization for One Lot
lot_df = df[df["Lot_ID"] == df["Lot_ID"].iloc[0]].sort_values("Timestamp")
source = ColumnDataSource(lot_df)

p = figure(title=f"Model 3: Competitive Pricing Over Time for Lot {lot_df['Lot_ID'].iloc[0]}",
           x_axis_type="datetime", width=800, height=400)
p.line(x="Timestamp", y="Competitive_Price", source=source, line_width=2, color="green", legend_label="Competitive Price")
p.circle(x="Timestamp", y="Competitive_Price", source=source, size=4, color="black")

p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price (₹)"
p.legend.location = "top_left"
show(p)


Loading BokehJS ...